In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [15]:
import logging

import pandas as pd

import core.config as cconfig
import core.finance as cofinanc
import dataflow.system as dtfsys
import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hpandas as hpandas
import helpers.hprint as hprint
import im_v2.ccxt.data.client as icdcl
import im_v2.common.data.client as icdc
import im_v2.common.universe as ivcu
import im_v2.crypto_chassis.data.client as iccdc
import market_data.market_data_example as mdmadaex

In [3]:
log_level = logging.INFO
hdbg.init_logger(verbosity=log_level)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

INFO: > cmd='/venv/lib/python3.8/site-packages/ipykernel_launcher.py -f /home/.local/share/jupyter/runtime/kernel-29d6707d-d1ce-4daa-9a84-e17e38ba2216.json'
-----------------------------------------------------------------------------
This code is not in sync with the container:
code_version='1.4.2' != container_version='1.4.0'
-----------------------------------------------------------------------------
You need to:
- merge origin/master into your branch with `invoke git_merge_master`
- pull the latest container with `invoke docker_pull`
INFO  # Git
  branch_name='CMTask4778_Investigate_bid_ask_data'
  hash='90127e64d'
  # Last commits:
    *   90127e64d Nina Lee Merge branch 'master' into CMTask4778_Investigate_bid_ask_data    (45 minutes ago) Mon Jul 17 10:12:19 2023  (HEAD -> CMTask4778_Investigate_bid_ask_data, origin/CMTask4778_Investigate_bid_ask_data)
    |\  
    | * 252cb7f11 Juraj Smeriga CmampTask4704_Create_test_case_suite_for_v2_broker_submit_twap_orders_2 (#4802) (    2 

# OHLCV market data

In [4]:
# Initialize the client.
data_version = "v2"
universe_version = "v4"
dataset = "ohlcv"
contract_type = "futures"
data_snapshot = "20220620"
im_client = icdcl.get_CcxtHistoricalPqByTileClient_example1(
    data_version,
    universe_version,
    dataset,
    contract_type,
    data_snapshot=data_snapshot,
)
# Set expected values.
full_symbols = ["binance::BTC_USDT", "binance::ADA_USDT"]
start_ts = pd.Timestamp("2022-05-01 13:00:00+00:00")
end_ts = pd.Timestamp("2022-05-01 13:05:00+00:00")
columns = None
filter_data_mode = "assert"
actual_df = im_client.read_data(
    full_symbols, start_ts, end_ts, columns, filter_data_mode
)

In [5]:
symbols = im_client.get_universe()[3:6]
display(symbols)

['binance::BTC_USDT', 'binance::EOS_USDT', 'binance::ETH_USDT']

In [6]:
asset_ids = im_client.get_asset_ids_from_full_symbols(symbols)
print(asset_ids)

[1467591036, 2061507978, 1464553467]


In [7]:
columns = None
column_remap = None
market_data = mdmadaex.get_HistoricalImClientMarketData_example1(
    im_client, asset_ids, columns, column_remap
)

In [8]:
# TODO(*): Document whether we need UTC.
start_ts = pd.Timestamp("2022-05-01 13:00:00+00:00")
end_ts = pd.Timestamp("2022-05-02 13:05:00+00:00")
ts_col_name = "knowledge_timestamp"
df = market_data.get_data_for_interval(
    start_ts,
    end_ts,
    ts_col_name,
    asset_ids,
)
hpandas.df_to_str(df, log_level=log_level)

,asset_id,full_symbol,open,high,low,close,volume,knowledge_timestamp,start_ts
2022-05-01 09:00:00-04:00,1464553467,binance::ETH_USDT,2760.0,2762.56,2759.13,2762.56,287.454,2022-06-24 11:10:10.287766+00:00,2022-05-01 08:59:00-04:00
2022-05-01 09:00:00-04:00,1467591036,binance::BTC_USDT,37935.7,37959.2,37930.7,37959.2,71.386,2022-06-24 05:47:16.075108+00:00,2022-05-01 08:59:00-04:00
2022-05-01 09:00:00-04:00,2061507978,binance::EOS_USDT,2.043,2.045,2.043,2.045,10408.8,2022-06-24 09:24:17.040013+00:00,2022-05-01 08:59:00-04:00
,...,...,...,...,...,...,...,...,...
2022-05-02 09:04:00-04:00,1464553467,binance::ETH_USDT,2794.29,2797.24,2794.29,2796.12,290.666,2022-06-24 11:10:10.287766+00:00,2022-05-02 09:03:00-04:00
2022-05-02 09:04:00-04:00,1467591036,binance::BTC_USDT,38505.1,38540.0,38505.0,38529.0,79.599,2022-06-24 05:47:16.075108+00:00,2022-05-02 09:03:00-04:00
2022-05-02 09:04:00-04:00,2061507978,binance::EOS_USDT,2.055,2.062,2.054,2.06,88356.6,2022-06-24 09:24:17.040013+00:00,2022-05-02 09:03:00-04:00


In [9]:
data_source_node = dtfsys.HistoricalDataSource(
    "load_prices",
    market_data,
    "end_ts",
    True,
    col_names_to_remove=["knowledge_timestamp", "start_ts", "full_symbol"],
)
data_source_node.set_fit_intervals([(start_ts, end_ts)])

In [10]:
data = data_source_node.fit()["df_out"]
hpandas.df_to_str(data, log_level=log_level)

close                             high                              low                             open                           volume                      
                          1464553467 1467591036 2061507978 1464553467 1467591036 2061507978 1464553467 1467591036 2061507978 1464553467 1467591036 2061507978 1464553467 1467591036 2061507978
2022-05-01 09:00:00-04:00    2762.56    37959.2      2.045    2762.56    37959.2      2.045    2759.13    37930.7      2.043     2760.0    37935.7      2.043    287.454     71.386    10408.8
2022-05-01 09:01:00-04:00    2764.73    37973.9      2.049     2765.3    37987.8       2.05    2762.55    37959.1      2.044    2762.55    37959.2      2.045    1008.01    146.115    66058.2
2022-05-01 09:02:00-04:00    2763.97    37957.5      2.047    2764.73    37974.0      2.049    2763.09    37953.3      2.046    2764.73    37973.9      2.048    583.906     78.208    20788.0
                                 ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...
2022-05-02 09:03:00-04:00    2794.29    38505.0      2.055    2796.36    38518.9      2.055    2793.79    38500.0      2.049    2795.95    38518.9      2.051    611.852     85.365   120724.6
2022-05-02 09:04:00-04:00    2796.12    38529.0       2.06    2797.24    38540.0      2.062    2794.29    38505.0      2.054    2794.29    38505.1      2.055    290.666     79.599    88356.6
2022-05-02 09:05:00-04:00    2797.04    38538.1      2.061    2797.66    38540.9      2.061    2796.11    38528.0      2.059    2796.11    38529.0      2.061    240.935     57.524    34581.6

# Stitched OHLCV and bid/ask data

## Data version is "v2"

In [11]:
universe_version = "v4"
contract_type = "futures"
data_snapshot = "20220707"
#
dataset1 = "ohlcv"
im_client1 = iccdc.get_CryptoChassisHistoricalPqByTileClient_example1(
    universe_version,
    dataset1,
    contract_type,
    data_snapshot,
)
#
dataset2 = "bid_ask"
im_client2 = iccdc.get_CryptoChassisHistoricalPqByTileClient_example1(
    universe_version,
    dataset2,
    contract_type,
    data_snapshot,
)
#
asset_ids = [1467591036, 1464553467]
columns = None
column_remap = None
wall_clock_time = None
filter_data_mode = "assert"
#
im_client_market_data1 = mdmadaex.get_HistoricalImClientMarketData_example1(
    im_client1,
    asset_ids,
    columns,
    column_remap,
    wall_clock_time=wall_clock_time,
    filter_data_mode=filter_data_mode,
)
im_client_market_data2 = mdmadaex.get_HistoricalImClientMarketData_example1(
    im_client2,
    asset_ids,
    columns,
    column_remap,
    wall_clock_time=wall_clock_time,
    filter_data_mode=filter_data_mode,
)
market_data = mdmadaex.get_HorizontalStitchedMarketData_example1(
    im_client_market_data1,
    im_client_market_data2,
    asset_ids,
    columns,
    column_remap,
    wall_clock_time=wall_clock_time,
    filter_data_mode=filter_data_mode,
)

In [12]:
start_ts = pd.Timestamp("2022-05-01 13:00:00+00:00")
end_ts = pd.Timestamp("2022-05-02 13:05:00+00:00")
ts_col_name = "knowledge_timestamp"
df = market_data.get_data_for_interval(
    start_ts,
    end_ts,
    ts_col_name,
    asset_ids,
)
hpandas.df_to_str(df, log_level=log_level)

,asset_id,full_symbol,open,high,low,close,volume,vwap,number_of_trades,twap,knowledge_timestamp,start_ts,bid_price,bid_size,ask_price,ask_size
2022-05-01 09:00:00-04:00,1464553467,binance::ETH_USDT,2762.55,2765.3,2762.55,2764.73,1008.01,2764.2581,490.0,2764.1087,2022-06-20 09:49:40.140622+00:00,2022-05-01 08:59:00-04:00,2764.152123,2436.613,2764.003005,909.711
2022-05-01 09:00:00-04:00,1467591036,binance::BTC_USDT,37959.2,37987.8,37959.1,37973.9,146.115,37974.469,694.0,37974.598,2022-06-20 09:48:46.910826+00:00,2022-05-01 08:59:00-04:00,37973.770035,228.977,37974.633461,175.528
2022-05-01 09:01:00-04:00,1464553467,binance::ETH_USDT,2764.73,2764.73,2763.09,2763.97,584.206,2763.9137,302.0,2763.8355,2022-06-20 09:49:40.140622+00:00,2022-05-01 09:00:00-04:00,2763.800863,2165.185,2763.885699,862.162
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-02 09:03:00-04:00,1467591036,binance::BTC_USDT,38505.1,38540.0,38505.0,38529.0,79.599,38525.57,571.0,38525.125,2022-06-20 09:48:46.910826+00:00,2022-05-02 09:02:00-04:00,38527.300859,344.537,38523.1902,160.702
2022-05-02 09:04:00-04:00,1464553467,binance::ETH_USDT,2796.11,2797.66,2796.11,2797.04,240.935,2797.0636,231.0,2796.899,2022-06-20 09:49:40.140622+00:00,2022-05-02 09:03:00-04:00,2796.875836,1206.289,2796.964358,1318.165
2022-05-02 09:04:00-04:00,1467591036,binance::BTC_USDT,38529.0,38540.9,38528.0,38538.1,57.524,38533.817,474.0,38533.746,2022-06-20 09:48:46.910826+00:00,2022-05-02 09:03:00-04:00,38532.121083,252.337,38535.484828,235.788


In [13]:
data_source_node = dtfsys.HistoricalDataSource(
    "load_prices",
    market_data,
    "end_ts",
    True,
    col_names_to_remove=["knowledge_timestamp", "start_ts", "full_symbol"],
)
data_source_node.set_fit_intervals([(start_ts, end_ts)])

In [14]:
data = data_source_node.fit()["df_out"]
hpandas.df_to_str(data, log_level=log_level)

ask_price                 ask_size               bid_price                 bid_size                 close                  high                   low            number_of_trades                  open                  twap                volume                  vwap           
                            1464553467    1467591036 1464553467 1467591036   1464553467    1467591036 1464553467 1467591036 1464553467 1467591036 1464553467 1467591036 1464553467 1467591036       1464553467 1467591036 1464553467 1467591036 1464553467 1467591036 1464553467 1467591036 1464553467 1467591036
2022-05-01 09:00:00-04:00  2764.003005  37974.633461    909.711    175.528  2764.152123  37973.770035   2436.613    228.977    2764.73    37973.9     2765.3    37987.8    2762.55    37959.1            490.0      694.0    2762.55    37959.2  2764.1087  37974.598    1008.01    146.115  2764.2581  37974.469
2022-05-01 09:01:00-04:00  2763.885699   37963.92703    862.162    177.092  2763.800863  37962.984408   2165.185    191.785    2763.97    37957.5    2764.73    37974.0    2763.09    37953.3            302.0      471.0    2764.73    37973.9  2763.8355  37962.282    584.206     78.208  2763.9137   37964.79
2022-05-01 09:02:00-04:00  2763.430308  37957.112679     864.03    116.318  2763.207318  37955.899564   1951.813     169.22    2762.42    37954.0    2763.98    37957.6    2762.36    37945.6            265.0      328.0    2763.97    37957.6  2763.3025  37954.937    466.446     44.477  2763.2716  37954.824
                                   ...           ...        ...        ...          ...           ...        ...        ...        ...        ...        ...        ...        ...        ...              ...        ...        ...        ...        ...        ...        ...        ...        ...        ...
2022-05-02 09:03:00-04:00  2796.219035    38523.1902   1543.286    160.702  2795.911376  38527.300859   1344.882    344.537    2796.12    38529.0    2797.24    38540.0    2794.29    38505.0            334.0      571.0    2794.29    38505.1  2795.9478  38525.125    290.666     79.599  2795.8491   38525.57
2022-05-02 09:04:00-04:00  2796.964358  38535.484828   1318.165    235.788  2796.875836  38532.121083   1206.289    252.337    2797.04    38538.1    2797.66    38540.9    2796.11    38528.0            231.0      474.0    2796.11    38529.0   2796.899  38533.746    240.935     57.524  2797.0636  38533.817
2022-05-02 09:05:00-04:00  2798.602739  38547.402691     940.77     188.32  2798.497657  38545.895797   1281.511    297.967    2799.29    38550.0    2799.42    38554.4    2797.05    38534.0            585.0      625.0    2797.05    38538.1  2798.6345   38547.26   1046.668     110.09  2798.6133  38548.878

## Data version is "v3"

### Config

In [6]:
config = {
    # TODO(Grisha): pick the correct bid/ask dataset, see
    # CmTask4778 "Choose bid-ask dataset".
    "signature": "periodic_daily.airflow.downloaded_1sec.parquet.bid_ask.futures.v3.crypto_chassis.binance.v1_0_0",
    "start_timestamp": pd.Timestamp("2023-01-01 04:10:00", tz="UTC"),
    "end_timestamp": pd.Timestamp("2023-01-01 04:13:00", tz="UTC"),
    "full_symbols": ["binance::ETH_USDT"],
    "columns": {
        "raw_data_columns": [
            "bid_price_l1",
            "ask_price_l1",
            "bid_size_l1",
            "ask_size_l1",
            "exchange_id",
            "currency_pair",
        ],
        "bid_ask_cols": ["bid_size", "ask_size", "bid_price", "ask_price"],
        "column_remap": {
            "bid_price_l1": "bid_price",
            "ask_price_l1": "ask_price",
            "bid_size_l1": "bid_size",
            "ask_size_l1": "ask_size",
        },
        "ts_col_name": "timestamp",
        "full_symbol": "full_symbol",
        "currency_pair": "currency_pair",
    },
    "resample": {
        "rule": "T",
        "vwap_groups": [],
        "resample_kwargs": {
            "closed": "right",
            "label": "right",
        },
    },
    "ohlcv_im_client_config": {
        "data_version": "v3",
        "universe_version": "v7",
        "dataset": "ohlcv",
        "contract_type": "futures",
        # Data snapshot is not applicable for data version = "v3".
        "data_snapshot": "",
    },
}
config = cconfig.Config().from_dict(config)
print(config)

signature: periodic_daily.airflow.downloaded_1sec.parquet.bid_ask.futures.v3.crypto_chassis.binance.v1_0_0
start_timestamp: 2023-01-01 04:10:00+00:00
end_timestamp: 2023-01-01 04:13:00+00:00
full_symbols: ['binance::ETH_USDT']
columns: 
  raw_data_columns: ['bid_price_l1', 'ask_price_l1', 'bid_size_l1', 'ask_size_l1', 'exchange_id', 'currency_pair']
  bid_ask_cols: ['bid_size', 'ask_size', 'bid_price', 'ask_price']
  column_remap: 
    bid_price_l1: bid_price
    ask_price_l1: ask_price
    bid_size_l1: bid_size
    ask_size_l1: ask_size
  ts_col_name: timestamp
  full_symbol: full_symbol
  currency_pair: currency_pair
resample: 
  rule: T
  vwap_groups: []
  resample_kwargs: 
    closed: right
    label: right
ohlcv_im_client_config: 
  data_version: v3
  universe_version: v7
  dataset: ohlcv
  contract_type: futures
  data_snapshot: 


### Load raw data

In [9]:
reader = icdc.RawDataReader(config["signature"])
raw_data = reader.read_data(config["start_timestamp"], config["end_timestamp"])
raw_data.head()

INFO  Loading dataset schema file: /app/amp/data_schema/dataset_schema_versions/dataset_schema_v3.json
INFO  Loaded dataset schema version v3
INFO  Loading dataset schema file: /app/amp/data_schema/dataset_schema_versions/dataset_schema_v3.json
INFO  Loaded dataset schema version v3
INFO  Loading dataset schema file: /app/amp/data_schema/dataset_schema_versions/dataset_schema_v3.json
INFO  Loaded dataset schema version v3


,timestamp,bid_price_l1,bid_size_l1,bid_price_l2,bid_size_l2,bid_price_l3,bid_size_l3,bid_price_l4,bid_size_l4,bid_price_l5,bid_size_l5,bid_price_l6,bid_size_l6,bid_price_l7,bid_size_l7,bid_price_l8,bid_size_l8,bid_price_l9,bid_size_l9,bid_price_l10,bid_size_l10,ask_price_l1,ask_size_l1,ask_price_l2,ask_size_l2,ask_price_l3,ask_size_l3,ask_price_l4,ask_size_l4,ask_price_l5,ask_size_l5,ask_price_l6,ask_size_l6,ask_price_l7,ask_size_l7,ask_price_l8,ask_size_l8,ask_price_l9,ask_size_l9,ask_price_l10,ask_size_l10,exchange_id,knowledge_timestamp,currency_pair,year,month
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-01-01 04:10:00+00:00,1672546200,0.2448,449353.0,0.2447,233274.0,0.2446,408518.0,0.2445,477091.0,0.2444,840481.0,0.2443,271565.0,0.2442,434058.0,0.2441,355113.0,0.2440,379136.0,0.2439,122893.0,0.2449,242171.0,0.2450,478349.0,0.2451,560432.0,0.2452,530450.0,0.2453,464416.0,0.2454,664156.0,0.2455,662064.0,0.2456,333394.0,0.2457,358184.0,0.2458,204975.0,binance,2023-01-02 11:01:32.565350+00:00,ADA_USDT,2023,1
2023-01-01 04:10:01+00:00,1672546201,0.2448,423689.0,0.2447,257739.0,0.2446,408518.0,0.2445,477091.0,0.2444,840481.0,0.2443,270750.0,0.2442,434058.0,0.2441,355928.0,0.2440,379136.0,0.2439,122893.0,0.2449,243407.0,0.2450,478350.0,0.2451,563432.0,0.2452,527450.0,0.2453,464416.0,0.2454,664156.0,0.2455,662064.0,0.2456,333394.0,0.2457,358184.0,0.2458,204975.0,binance,2023-01-02 11:01:32.565350+00:00,ADA_USDT,2023,1
2023-01-01 04:10:02+00:00,1672546202,0.2448,409915.0,0.2447,257739.0,0.2446,408518.0,0.2445,477091.0,0.2444,811516.0,0.2443,299715.0,0.2442,434058.0,0.2441,348107.0,0.2440,379136.0,0.2439,122893.0,0.2449,243410.0,0.2450,453641.0,0.2451,563432.0,0.2452,527450.0,0.2453,464416.0,0.2454,641708.0,0.2455,662064.0,0.2456,333394.0,0.2457,358184.0,0.2458,204975.0,binance,2023-01-02 11:01:32.565350+00:00,ADA_USDT,2023,1
2023-01-01 04:10:03+00:00,1672546203,0.2447,83889.0,0.2446,198839.0,0.2445,397830.0,0.2444,859265.0,0.2443,217412.0,0.2442,476987.0,0.2441,301402.0,0.2440,409614.0,0.2439,214470.0,0.2438,188923.0,0.2448,445134.0,0.2449,534767.0,0.2450,422352.0,0.2451,614626.0,0.2452,466019.0,0.2453,478509.0,0.2454,723314.0,0.2455,493945.0,0.2456,191930.0,0.2457,262738.0,binance,2023-01-02 11:01:32.565350+00:00,ADA_USDT,2023,1
2023-01-01 04:10:04+00:00,1672546204,0.2447,83433.0,0.2446,195195.0,0.2445,406372.0,0.2444,858305.0,0.2443,230371.0,0.2442,555486.0,0.2441,298402.0,0.2440,447960.0,0.2439,217345.0,0.2438,194519.0,0.2448,347891.0,0.2449,535871.0,0.2450,428417.0,0.2451,620477.0,0.2452,454578.0,0.2453,486573.0,0.2454,700891.0,0.2455,491964.0,0.2456,191930.0,0.2457,262738.0,binance,2023-01-02 11:01:32.565350+00:00,ADA_USDT,2023,1


In [10]:
data = raw_data[config["columns"]["raw_data_columns"]].rename(
    columns=config["columns"]["column_remap"].to_dict()
)
data

,bid_price,ask_price,bid_size,ask_size,exchange_id,currency_pair
timestamp,,,,,,
2023-01-01 04:10:00+00:00,0.2448,0.2449,449353.0,242171.0,binance,ADA_USDT
2023-01-01 04:10:01+00:00,0.2448,0.2449,423689.0,243407.0,binance,ADA_USDT
2023-01-01 04:10:02+00:00,0.2448,0.2449,409915.0,243410.0,binance,ADA_USDT
2023-01-01 04:10:03+00:00,0.2447,0.2448,83889.0,445134.0,binance,ADA_USDT
2023-01-01 04:10:04+00:00,0.2447,0.2448,83433.0,347891.0,binance,ADA_USDT
...,...,...,...,...,...,...
2023-01-01 04:12:56+00:00,0.3375,0.3376,325247.0,331154.7,binance,XRP_USDT
2023-01-01 04:12:57+00:00,0.3375,0.3376,325160.9,333507.1,binance,XRP_USDT
2023-01-01 04:12:58+00:00,0.3375,0.3376,338087.4,330144.1,binance,XRP_USDT


### Resample

In [ ]:
# TODO(Grisha): Ideally load already resampled to 1 minute bid-ask data,
# see CmTask4792 "Confirm bid / ask resampling".

In [13]:
col_filter = lambda col: any(
    [x in col for x in config["columns"]["bid_ask_cols"]]
)
cols_dict = {col: col for col in filter(col_filter, data.columns)}
resampling_groups = [(cols_dict, "mean", {})]
data_resampled = []
for currency_pair in data[config["columns"]["currency_pair"]].unique():
    data_single = data[data[config["columns"]["currency_pair"]] == currency_pair]
    data_resampled_single = cofinanc.resample_bars(
        data_single, **config["resample"], resampling_groups=resampling_groups
    )
    data_resampled_single[config["columns"]["currency_pair"]] = currency_pair
    data_resampled.append(data_resampled_single)
data_resampled = pd.concat(data_resampled)
data_resampled.head(3)

,bid_price,ask_price,bid_size,ask_size,currency_pair
timestamp,,,,,
2023-01-01 04:10:00+00:00,0.244800,0.244900,449353.000000,242171.000000,ADA_USDT
2023-01-01 04:11:00+00:00,0.244698,0.244798,120788.116667,273614.966667,ADA_USDT
2023-01-01 04:12:00+00:00,0.244700,0.244800,212803.866667,76249.433333,ADA_USDT


In [16]:
data_resampled["exchange"] = "binance"
# Add a full symbol column.
data_resampled[config["columns"]["full_symbol"]] = ivcu.build_full_symbol(
    data_resampled["exchange"], data_resampled[config["columns"]["currency_pair"]]
)
data_resampled.head(3)

,bid_price,ask_price,bid_size,ask_size,currency_pair,exchange,full_symbol
timestamp,,,,,,,
2023-01-01 04:10:00+00:00,0.244800,0.244900,449353.000000,242171.000000,ADA_USDT,binance,binance::ADA_USDT
2023-01-01 04:11:00+00:00,0.244698,0.244798,120788.116667,273614.966667,ADA_USDT,binance,binance::ADA_USDT
2023-01-01 04:12:00+00:00,0.244700,0.244800,212803.866667,76249.433333,ADA_USDT,binance,binance::ADA_USDT


### Stiched market data

In [18]:
# Initialize `DataFrameImClient`.
# TODO(Grisha): Check if we can use `HistoricalPqByTileClient`,
# see CmTask4791 "Check if we can load 1 minute bid-ask data using ImClient".
bid_ask_im_client = icdc.DataFrameImClient(
    data_resampled, universe=config["full_symbols"]
)
#
filter_data_mode = "assert"
columns = None
bid_ask_im_client.read_data(
    config["full_symbols"],
    config["start_timestamp"],
    config["end_timestamp"],
    columns=columns,
    filter_data_mode=filter_data_mode,
)

,bid_price,ask_price,bid_size,ask_size,currency_pair,exchange,full_symbol
timestamp,,,,,,,
2023-01-01 04:10:00+00:00,1192.760000,1192.770000,0.877000,268.159000,ETH_USDT,binance,binance::ETH_USDT
2023-01-01 04:11:00+00:00,1192.876833,1192.886833,72.744667,196.281517,ETH_USDT,binance,binance::ETH_USDT
2023-01-01 04:12:00+00:00,1193.056833,1193.066833,57.989617,140.999017,ETH_USDT,binance,binance::ETH_USDT
2023-01-01 04:13:00+00:00,1193.280333,1193.290333,163.515767,108.915950,ETH_USDT,binance,binance::ETH_USDT


In [20]:
# Initialize `CcxtHistoricalPqByTileClient`.
ohlcv_im_client = icdcl.get_CcxtHistoricalPqByTileClient_example1(
    **config["ohlcv_im_client_config"]
)
#
asset_ids = list(
    ivcu.build_numerical_to_string_id_mapping(config["full_symbols"]).keys()
)
column_remap = None
wall_clock_time = None
im_client_market_data1 = mdmadaex.get_HistoricalImClientMarketData_example1(
    bid_ask_im_client,
    asset_ids,
    columns,
    column_remap,
    wall_clock_time=wall_clock_time,
)
im_client_market_data2 = mdmadaex.get_HistoricalImClientMarketData_example1(
    ohlcv_im_client,
    asset_ids,
    columns,
    column_remap,
    wall_clock_time=wall_clock_time,
    filter_data_mode=filter_data_mode,
)
stiched_mdata = mdmadaex.get_HorizontalStitchedMarketData_example1(
    im_client_market_data1,
    im_client_market_data2,
    asset_ids,
    columns,
    column_remap,
    wall_clock_time=wall_clock_time,
    filter_data_mode=filter_data_mode,
)

WARNING Data that is downloaded before a bar ends accounts for=28 / 1911855 = 0.00%


In [21]:
stiched_mdata.get_data_for_interval(
    config["start_timestamp"],
    config["end_timestamp"],
    config["columns"]["ts_col_name"],
    asset_ids,
)

,asset_id,bid_price,ask_price,bid_size,ask_size,currency_pair,exchange,full_symbol,start_ts,open,high,low,close,volume,knowledge_timestamp
end_ts,,,,,,,,,,,,,,,
2022-12-31 23:10:00-05:00,1464553467,1192.760000,1192.770000,0.877000,268.159000,ETH_USDT,binance,binance::ETH_USDT,2022-12-31 23:09:00-05:00,1193.53,1193.53,1192.71,1192.71,1668.269,2023-01-02 00:17:01.141222+00:00
2022-12-31 23:11:00-05:00,1464553467,1192.876833,1192.886833,72.744667,196.281517,ETH_USDT,binance,binance::ETH_USDT,2022-12-31 23:10:00-05:00,1192.72,1193.09,1192.71,1193.09,2362.057,2023-01-02 00:17:01.141222+00:00
2022-12-31 23:12:00-05:00,1464553467,1193.056833,1193.066833,57.989617,140.999017,ETH_USDT,binance,binance::ETH_USDT,2022-12-31 23:11:00-05:00,1193.08,1193.19,1192.85,1193.10,821.965,2023-01-02 00:17:01.141222+00:00


In [22]:
nid = "read_data"
ts_col_name = "end_ts"
multiindex_output = True
data_source_node = dtfsys.HistoricalDataSource(
    nid,
    stiched_mdata,
    ts_col_name,
    multiindex_output,
    col_names_to_remove=["knowledge_timestamp", "start_ts", "full_symbol"],
)
data_source_node.set_fit_intervals(
    [(config["start_timestamp"], config["end_timestamp"])]
)
df = data_source_node.fit()["df_out"]
df.head()

,ask_price,ask_size,bid_price,bid_size,close,currency_pair,exchange,high,low,open,volume
,1464553467,1464553467,1464553467,1464553467,1464553467,1464553467,1464553467,1464553467,1464553467,1464553467,1464553467
end_ts,,,,,,,,,,,
2022-12-31 23:10:00-05:00,1192.770000,268.159000,1192.760000,0.877000,1192.71,ETH_USDT,binance,1193.53,1192.71,1193.53,1668.269
2022-12-31 23:11:00-05:00,1192.886833,196.281517,1192.876833,72.744667,1193.09,ETH_USDT,binance,1193.09,1192.71,1192.72,2362.057
2022-12-31 23:12:00-05:00,1193.066833,140.999017,1193.056833,57.989617,1193.10,ETH_USDT,binance,1193.19,1192.85,1193.08,821.965
2022-12-31 23:13:00-05:00,1193.290333,108.915950,1193.280333,163.515767,1193.38,ETH_USDT,binance,1193.50,1193.10,1193.10,282.627
